# Building an image retrieval system with deep features


# Fire up GraphLab Create

In [130]:
import graphlab

# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [131]:
image_train = graphlab.SFrame('image_train_data/')

# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [ ]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [132]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [133]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [134]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:20]
cat['image'].show()

In [135]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 2       | 0.0498753   | 67.104ms     |
PROGRESS: | Done         |         | 100         | 413.168ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5
1,424,0.0,1
1,43978,35.5389067307,2
1,13387,35.8600902579,3
1,46659,36.7698429143,4
1,21889,37.2279843858,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [136]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [137]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 2       | 0.0498753   | 27.411ms     |
PROGRESS: | Done         |         | 100         | 434.473ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [138]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [11]:
car = image_train[8:9]
car['image'].show()

In [12]:
get_images_from_ids(knn_model.query(car))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 47.437ms     |
PROGRESS: | Done         |         | 100         | 355.144ms    |
PROGRESS: +--------------+---------+-------------+--------------+


# Just for fun, let's create a lambda to find and show nearest neighbor images

In [13]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [14]:
show_neighbors(8)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 42.601ms     |
PROGRESS: | Done         |         | 100         | 373.989ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [15]:
show_neighbors(111)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 35.254ms     |
PROGRESS: | Done         |         | 100         | 583.679ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [16]:
image_train[11:12][
    'image'
].show()

# Quiz

## computing summary statistics of the data. 

In [139]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


### Creating category-specific image retrieval models

In [140]:
sf_dog = image_train.filter_by('dog','label')

In [141]:
sf_dog.head()

id,image,label,deep_features,image_array
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
177,Height: 32 Width: 32,dog,"[0.0, 1.45965671539, 0.0,0.422992348671, 0.0, ...","[55.0, 75.0, 42.0, 51.0,76.0, 37.0, 57.0, 83.0, ..."
424,Height: 32 Width: 32,dog,"[0.942399680614, 0.0,0.220352768898, 0.0, ...","[60.0, 35.0, 18.0, 63.0,49.0, 38.0, 66.0, 56.0, ..."
462,Height: 32 Width: 32,dog,"[1.43462562561, 0.0, 0.0,0.0, 0.0, 0.0, ...","[86.0, 69.0, 75.0, 57.0,41.0, 48.0, 46.0, 35.0, ..."
542,Height: 32 Width: 32,dog,"[0.451547086239, 0.0,0.0, 0.0, 0.0, 0.0, ...","[196.0, 174.0, 113.0,140.0, 117.0, 65.0, 8 ..."
573,Height: 32 Width: 32,dog,"[0.592360973358, 0.0,0.0, 0.0, 0.0, 0.0, ...","[153.0, 103.0, 52.0,151.0, 102.0, 49.0, ..."
851,Height: 32 Width: 32,dog,"[0.690123438835, 0.0,0.0, 0.0, 0.305860161 ...","[39.0, 6.0, 4.0, 53.0,23.0, 24.0, 57.0, 37.0, ..."
919,Height: 32 Width: 32,dog,"[0.0, 0.177558660507,0.139396846294, 0.0, ...","[29.0, 43.0, 4.0, 24.0,35.0, 6.0, 24.0, 37.0, ..."
1172,Height: 32 Width: 32,dog,"[0.517601490021, 0.0,1.96418333054, 0.0, 0.0, ...","[182.0, 180.0, 197.0,196.0, 192.0, 209.0, ..."


In [142]:
sf_cat = image_train.filter_by('cat','label')

In [145]:
sf_auto = image_train.filter_by('automobile','label')
sf_auto.head()

id,image,label,deep_features,image_array
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
302,Height: 32 Width: 32,automobile,"[0.583938002586, 0.0,0.0, 0.0, 0.0, 0.0, ...","[64.0, 52.0, 37.0, 85.0,60.0, 40.0, 92.0, 66.0, ..."
312,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.392823398113, 0.0, ...","[124.0, 126.0, 113.0,124.0, 126.0, 113.0, ..."
323,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,4.42310428619, ...","[241.0, 241.0, 241.0,238.0, 238.0, 238.0, ..."
536,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.42903900146, 0.0, ...","[164.0, 154.0, 154.0,128.0, 119.0, 120.0, ..."
593,Height: 32 Width: 32,automobile,"[1.65033948421, 0.0, 0.0,0.0, 0.0, 0.0, ...","[231.0, 222.0, 227.0,232.0, 217.0, 221.0, ..."
962,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.39552795887, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
997,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.04085636139, 0.0, ...","[145.0, 148.0, 157.0,131.0, 134.0, 145.0, ..."
1421,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.359612941742, ...","[114.0, 95.0, 33.0,118.0, 98.0, 26.0, 91.0, ..."


In [144]:
sf_bird = image_train.filter_by('bird','label')

In [146]:
dog_model = graphlab.nearest_neighbors.create(sf_dog,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [147]:
cat_model = graphlab.nearest_neighbors.create(sf_cat,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [148]:
auto_model = graphlab.nearest_neighbors.create(sf_auto,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [149]:
bird_model = graphlab.nearest_neighbors.create(sf_bird,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [150]:
image_test = graphlab.SFrame('image_test_data/')
graphlab.canvas.set_target('ipynb')
quest_image = image_test[0:1]
quest_image['image'].show()

In [163]:
cat_neighbors = get_images_from_ids(cat_model.query(quest_image))['label' == '16289']['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 46.394ms     |
PROGRESS: | Done         |         | 100         | 182.711ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [155]:
cat_model.query(quest_image)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 29.436ms     |
PROGRESS: | Done         |         | 100         | 172.526ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [164]:
image_train.filter_by(dog_model.query(quest_image)['reference_label'],'id')['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 63.719ms     |
PROGRESS: | Done         |         | 100         | 194.322ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [51]:
dog_model.query(quest_image)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 35.991ms     |
PROGRESS: | Done         |         | 100         | 167.775ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [166]:
image_train.filter_by(dog_model.query(quest_image)['reference_label'],'id')['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 37.988ms     |
PROGRESS: | Done         |         | 100         | 163.734ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [125]:
image_train.filter_by(cat_model.query(quest_image)['reference_label'],'id')['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 49.317ms     |
PROGRESS: | Done         |         | 100         | 177.94ms     |
PROGRESS: +--------------+---------+-------------+--------------+


### Mean

In [57]:
dog_model.query(quest_image)['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 32.328ms     |
PROGRESS: | Done         |         | 100         | 161.118ms    |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184156

In [58]:
cat_model.query(quest_image)['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 45.804ms     |
PROGRESS: | Done         |         | 100         | 185.061ms    |
PROGRESS: +--------------+---------+-------------+--------------+


36.15573070978294

## Computing nearest neighbors accuracy using SFrame operations:#

In [59]:
image_test_cat = image_test.filter_by('cat','label')

In [60]:
image_test_dog = image_test.filter_by('dog','label')

In [61]:
image_test_auto = image_test.filter_by('automobile','label')

In [62]:
image_test_bird = image_test.filter_by('bird','label')

In [63]:
dog_model.query()

TypeError: query() takes at least 2 arguments (1 given)

In [64]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 622.648ms    |
PROGRESS: | Done         | 509000  | 100         | 739.116ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [65]:
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 723.066ms    |
PROGRESS: | Done         | 509000  | 100         | 810.353ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [66]:
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 119000  | 24.8954     | 709.326ms    |
PROGRESS: | Done         | 478000  | 100         | 765.832ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [67]:
dog_auto_neighbors = auto_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 684.7ms      |
PROGRESS: | Done         | 509000  | 100         | 798.458ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [68]:
dog_auto_neighbors.head()

query_label,reference_label,distance,rank
0,33859,41.9579761457,1
1,2046,46.0021331807,1
2,19594,42.9462290692,1
3,11000,41.6866060048,1
4,19594,39.2269664935,1
5,49314,40.5845117698,1
6,40822,45.1067352961,1
7,44997,41.3221140974,1
8,33859,41.8244654995,1
9,33859,45.4976929401,1


In [69]:
combined_frame = graphlab.SFrame({'dog-dog':dog_dog_neighbors['distance'],
                                  'dog-cat':dog_cat_neighbors['distance'],
                                  'dog-bird':dog_bird_neighbors['distance'],
                                  'dog-auto':dog_auto_neighbors['distance'],
                                 })

In [70]:
combined_frame.head()

dog-auto,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


### Computing the number of correct predictions using 1-nearest neighbors for the dog class:

In [111]:
test_row = combined_frame[1:2]
test_row.show()
1<2
(test_row['dog-dog'] < test_row['dog-auto'])

dtype: int
Rows: 1
[1]

In [113]:
def is_dog_correct(row):
    if (row['dog-dog'][0] < row['dog-auto'][0]) and (row['dog-dog'][0] < row['dog-cat'][0]) and (row['dog-dog'][0] < row['dog-bird'][0]):
        return 1
    return 0

In [114]:
is_dog_correct(test_row)

1

In [118]:
def is_dog_correct1(row):
    if (row['dog-dog'] < row['dog-auto']) and (row['dog-dog'] < row['dog-cat']) and (row['dog-dog'] < row['dog-bird']):
        return 1
    return 0

In [167]:
combined_frame.apply(is_dog_correct1).sum()

678

In [171]:
len(sf_dog)

509

In [172]:
len(image_train)

2005

In [173]:
## for cat

In [ ]:
cat_dog_neighbors = cat_model.query(image_test_dog, k=1)
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
